In [ ]:
import copy
import pandas as pd
from nasbench import api
from tqdm.notebook import tqdm

nasbench = api.NASBench("nasbench_full.tfrecord")

In [ ]:
INPUT = "input"
OUTPUT = "output"
CONV3X3 = "conv3x3-bn-relu"
CONV1X1 = "conv1x1-bn-relu"
MAXPOOL3X3 = "maxpool3x3"

NUM_VERTICES = 7
MAX_EDGES = 9

EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2  # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2  # Input/output vertices are fixed

ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1]  # Binary adjacency matrix"

In [ ]:
# query max accuracy model from dataset
all_data = []
for key, data in tqdm(nasbench.fixed_statistics.items()):
    results = nasbench.query(
        api.ModelSpec(matrix=data["module_adjacency"], ops=data["module_operations"])
    )
    results = copy.deepcopy(results)
    results["hash"] = key
    del results["module_adjacency"]
    del results["module_operations"]
    results = [
        results["hash"],
        results["trainable_parameters"],
        results["training_time"],
        results["train_accuracy"],
        results["validation_accuracy"],
        results["test_accuracy"],
    ]
    all_data.append(results)

In [ ]:
df = pd.DataFrame(
    all_data,
    columns=[
        "Hash",
        "Trainable Parameters",
        "Training Time",
        "Train Accuracy",
        "Validation Accuracy",
        "Test Accuracy",
    ],
)

In [ ]:
df._get_numeric_data().min()

In [ ]:
df._get_numeric_data().max()

In [ ]:
df._get_numeric_data().median()

In [ ]:
df._get_numeric_data().mean()

In [ ]:
df._get_numeric_data().std()